# Tutorial 4: Generating time series

`synthcity` includes models targeting specific tabular modalities. One of the specific modalities is for __time series__. The general-purpose models can also be used for this task.

The main requirement of survival-analysis is to use a `TimeSeriesDataLoader`/`TimeSeriesSurvivalDataLoader` dataloader.

In [ ]:
# stdlib
import sys
import warnings

# synthcity absolute
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import TimeSeriesSurvivalDataLoader

log.add(sink=sys.stderr, level="INFO")
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
from synthcity.utils.datasets.time_series.pbc import PBCDataloader

(
    static_surv,
    temporal_surv,
    temporal_surv_horizons,
    outcome_surv,
) = PBCDataloader().load()
T, E = outcome_surv

horizons = [0.25, 0.5, 0.75]
time_horizons = np.quantile(T, horizons).tolist()

loader = TimeSeriesSurvivalDataLoader(
    temporal_data=temporal_surv,
    temporal_horizons=temporal_surv_horizons,
    static_data=static_surv,
    T=T,
    E=E,
    time_horizons=time_horizons,
)

loader.dataframe()

## List the available generative models

In [ ]:
# synthcity absolute
from synthcity.plugins import Plugins

Plugins(categories=["time_series"]).list()

## Load and train a generative model

In [ ]:
# synthcity absolute
from synthcity.plugins import Plugins

syn_model = Plugins().get("timegan")

syn_model.fit(loader)

## Generate new data using the model

In [ ]:
syn_model.generate(count=10).dataframe()

## Plot real-synthetic distributions

Compared to the general case, the survival-analysis data includes KM plots for real and synthetic data.

In [ ]:
# third party
import matplotlib.pyplot as plt

syn_model.plot(plt, loader)

plt.show()

## Benchmark the quality of plugins

For survival analysis, general purpose generators can be used as well.

In [ ]:
# synthcity absolute
from synthcity.benchmark import Benchmarks

score = Benchmarks.evaluate(
    [
        (f"test_{model}", model, {})
        for model in ["adsgan", "timegan", "probabilistic_ar"]
    ],
    loader,
    synthetic_size=1000,
    repeats=2,
    task_type="time_series_survival", # time_series_survival or time_series
)

In [ ]:
Benchmarks.print(score)

# 